Content based recommendation algorithm, based on what the user likes will pick other items which have similar features and recommends us. It is totally subjective to only one individual user

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens-100k-dataset/ml-100k/u.data
/kaggle/input/movielens-100k-dataset/ml-100k/ub.test
/kaggle/input/movielens-100k-dataset/ml-100k/mku.sh
/kaggle/input/movielens-100k-dataset/ml-100k/u3.base
/kaggle/input/movielens-100k-dataset/ml-100k/u.info
/kaggle/input/movielens-100k-dataset/ml-100k/u.item
/kaggle/input/movielens-100k-dataset/ml-100k/u1.base
/kaggle/input/movielens-100k-dataset/ml-100k/allbut.pl
/kaggle/input/movielens-100k-dataset/ml-100k/u1.test
/kaggle/input/movielens-100k-dataset/ml-100k/u2.test
/kaggle/input/movielens-100k-dataset/ml-100k/ua.test
/kaggle/input/movielens-100k-dataset/ml-100k/README
/kaggle/input/movielens-100k-dataset/ml-100k/u3.test
/kaggle/input/movielens-100k-dataset/ml-100k/u.user
/kaggle/input/movielens-100k-dataset/ml-100k/u.genre
/kaggle/input/movielens-100k-dataset/ml-100k/u5.test
/kaggle/input/movielens-100k-dataset/ml-100k/u4.base
/kaggle/input/movielens-100k-dataset/ml-100k/ua.base
/kaggle/input/movielens-100k-dataset/ml-100k/u.occ

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
movie_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
moviesdf = pd.read_csv('../input/movielens-100k-dataset/ml-100k/u.item', sep='|', names=movie_cols, usecols=range(5),encoding='latin-1')
moviesdf.drop(columns = ['release_date', 'video_release_date', 'imdb_url'], axis = 1, inplace = True)
moviesdf.head()

In [2]:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('../input/movielens-100k-dataset/ml-100k/u.item', sep='|', names=i_cols,encoding='latin-1')

items.head()


,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [3]:
print(items.shape)

(1682, 24)


In [45]:
listofgenres = pd.read_csv('../input/movielens-100k-dataset/ml-100k/u.genre',sep='|', names = ['genre', 'id']) 
listofgenres

,genre,id
0,unknown,0
1,Action,1
2,Adventure,2
3,Animation,3
4,Children's,4
5,Comedy,5
6,Crime,6
7,Documentary,7
8,Drama,8
9,Fantasy,9


Here we have the genres split into different features with 1 meaning True and 0 meaning False. We do not want that, so we are changing it to a single column 'Genre' and populating with all values that had 1

In [4]:
def get_genre(row):
    genre_list = []
    for c in items.columns:
        if row[c]==1 and c != 'movie id':
            genre_list.append(c)
    return ' '.join(genre_list)

In [5]:
 items['genre']=items.apply(get_genre, axis=1)

In [6]:
items.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,genre
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,Animation Children's Comedy
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,Action Adventure Thriller
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Thriller
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Action Comedy Drama
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Crime Drama Thriller


Delete the individual genre columns 

In [7]:
items.drop(columns = ['unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], axis = 1, inplace = True)
items.head()

,movie id,movie title,release date,video release date,IMDb URL,genre
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,Animation Children's Comedy
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,Action Adventure Thriller
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,Thriller
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,Action Comedy Drama
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),Crime Drama Thriller


In [9]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(items["genre"])
cos_sim = cosine_similarity(count_matrix) 

In [41]:
def recommend(input_movie):
    reclist = []
    movie_index = items.index[items['movie title']== input_movie].values[0]
    similar_movies =  list(enumerate(cos_sim[movie_index]))
    sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)
    i=0
    for element in sorted_similar_movies:
        reclist.append(items[items.index == element[0]]['movie title'].values[0])
        i+=1
        if i>10:
            break
    if input_movie in reclist:
        reclist.remove(input_movie)
    for mov in reclist:
        print(mov)
#     return reclist

If our user likes the movie Grand Day Out, A (1992), then based on that input the following movies will be recommended.

In [42]:
recommend("Grand Day Out, A (1992)")

Wrong Trousers, The (1993)
Beavis and Butt-head Do America (1996)
Toy Story (1995)
Close Shave, A (1995)
Aladdin and the King of Thieves (1996)
Mighty Aphrodite (1995)
Birdcage, The (1996)
Brothers McMullen, The (1995)
To Wong Foo, Thanks for Everything! Julie Newmar (1995)
Billy Madison (1995)


In [43]:
recommend("Young Poisoner's Handbook, The (1995)")


Kansas City (1996)
Deceiver (1997)
Freeway (1996)
Keys to Tulsa (1997)
2 Days in the Valley (1996)
Curdled (1996)
Original Gangstas (1996)
Switchblade Sisters (1975)
Big Bang Theory, The (1994)
Gridlock'd (1997)
